In [67]:
import os
import nest_asyncio
from dotenv import load_dotenv

nest_asyncio.apply()
load_dotenv()

# Validate that your environment variables are loaded
print("AZURE_OPENAI_API_KEY:", os.getenv("AZURE_OPENAI_API_KEY")[:5] + "...")


AZURE_OPENAI_API_KEY: Ab2Vv...


In [68]:
# Agent names
BA = "BusinessAnalyst"
SE = "SoftwareEngineer"
PO = "ProductOwner"

# Define agent personas
business_analyst_persona = """
    You are a Business Analyst which will take the requirements from the user (also known as a 'customer') 
    and create a project plan for creating the requested app. The Business Analyst understands the user 
    requirements and creates detailed documents with requirements and costing. The documents should be 
    usable by the SoftwareEngineer as a reference for implementing the required features, and by the 
    Product Owner for reference to determine if the application delivered by the Software Engineer meets 
    all of the user's requirements.
    """

software_engineer_persona = """
    You are a Software Engineer, and your goal is create a web app using HTML and JavaScript by taking 
    into consideration all the requirements given by the Business Analyst. The application should 
    implement all the requested features. Deliver the code to the Product Owner for review when completed. You can also ask questions of the BusinessAnalyst to clarify any requirements that are unclear.
    """

product_owner_persona = """
    You are the Product Owner which will review the software engineer's code to ensure all user requirements 
    are completed. You are the guardian of quality, ensuring the final product meets all specifications. 
    IMPORTANT: Verify that the Software Engineer has shared the HTML code using the format ```html [code] ```. 
    This format is required for the code to be saved and pushed to GitHub. Once all client requirements 
    are completed and the code is properly formatted, reply with 'READY FOR USER APPROVAL'. If there are 
    missing features or formatting issues, you will need to send a request back to the SoftwareEngineer 
    or BusinessAnalyst with details of the defect.
    """

In [69]:
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from semantic_kernel.agents import ChatCompletionAgent

kernel = Kernel()
chat_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)
kernel.add_service(chat_service)

ba = ChatCompletionAgent(name=BA, instructions=business_analyst_persona, kernel=kernel)
se = ChatCompletionAgent(name=SE, instructions=software_engineer_persona, kernel=kernel)
po = ChatCompletionAgent(name=PO, instructions=product_owner_persona, kernel=kernel)


In [70]:
from semantic_kernel.functions import KernelFunctionFromPrompt
from semantic_kernel.agents.strategies.selection.kernel_function_selection_strategy import KernelFunctionSelectionStrategy
from semantic_kernel.contents import ChatHistoryTruncationReducer

# Define selection strategy
selection_function = KernelFunctionFromPrompt(
        function_name="selection",
        prompt=f"""Examine the provided RESPONSE and choose the next participant. State only the name of the chosen participant without 
        explanation. Never choose the participant named in the RESPONSE.
        
        Participants:
        - {BA}
        - {SE}
        - {PO}

        Rules:
        - If RESPONSE is USER input, it's {BA}'s turn.
        - If RESPONSE is by {BA}, it's {SE}'s turn.
        - If RESPONSE is by {SE}, it's {PO}'s turn.
        - If RESPONSE is by {PO} and contains 'READY FOR USER APPROVAL', wait for user input.
        - If RESPONSE is by {PO} and does NOT contain 'READY FOR USER APPROVAL', it's {SE}'s turn.

        RESPONSE:
        {{{{$lastmessage}}}}""",
            )

selection_strategy = KernelFunctionSelectionStrategy(
    initial_agent=ba,
    function=selection_function,
    kernel=kernel,
    history_variable_name="lastmessage",
    result_parser=lambda result: str(result.value[0]).strip() if result.value[0] is not None else SE,
    history_reducer=ChatHistoryTruncationReducer(target_count=3),
)


In [71]:
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents.utils.author_role import AuthorRole

class ApprovalTerminationStrategy(TerminationStrategy):
    async def should_agent_terminate(self, agent, history):
        for message in history:
            if message.role == AuthorRole.USER and "APPROVED" in message.content.upper():
                return True
        return False


In [72]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.contents.chat_message_content import ChatMessageContent

group_chat = AgentGroupChat(
    agents=[ba, se, po],
    selection_strategy=selection_strategy,
    termination_strategy=ApprovalTerminationStrategy()
)

initial_input = "I want to develop app which will provide me calculator. Keep it very simple. And get final approval from manager."
conversation = []
first_msg = ChatMessageContent(role=AuthorRole.USER, content=initial_input)
await group_chat.add_chat_message(first_msg)
conversation.append({"role": "user", "content": initial_input})


In [73]:
# response_iterator = group_chat.invoke().__aiter__()

In [74]:
# response = await response_iterator.__anext__()

# if response:
#     print(f"{response.role} - {response.name}: {response.content}")
#     conversation.append({"role": response.role, "content": response.content})


In [75]:
# response.role

In [76]:
# "READY FOR USER APPROVAL" in response.content.upper()

In [77]:
# Step 1: Start the initial agent chat until approval is needed
async for response in group_chat.invoke():
    if response is None:
        continue

    print(f"{response.role} - {response.name}: {response.content}")
    conversation.append({"role": response.role, "content": response.content})

    # Pause if assistant requests user approval
    if response.role == AuthorRole.ASSISTANT and "READY FOR USER APPROVAL" in response.content.upper():
        break  # exit this loop cleanly before injecting user message

# ✅ At this point, the previous `invoke()` loop has exited and lock is released

# Step 2: Wait for user input
user_input = input("💬 Type 'APPROVED' to approve: ")
approved_msg = ChatMessageContent(role=AuthorRole.USER, content=user_input)
await group_chat.add_chat_message(approved_msg)

# Step 3: Run next loop for agent response to user input
async for response in group_chat.invoke():
    if response is None:
        continue

    print(f"{response.role} - {response.name}: {response.content}")
    conversation.append({"role": response.role, "content": response.content})

    if "APPROVED" in response.content.upper():
        print("✅ Approval acknowledged by agent.")
        break  # Optional: stop if approval is acknowledged


assistant - BusinessAnalyst: ### Project Plan Document: Calculator App Development

---

#### **1. Project Details**
- **Project Name:** Simple Calculator App
- **Project Objective:** Create a basic and user-friendly calculator application that supports fundamental arithmetic operations (addition, subtraction, multiplication, and division).
- **Project Scope:** This app will provide users with a minimalistic interface for performing arithmetic calculations. Advanced features (e.g., scientific or graphing functionalities) will not be included to maintain simplicity.

---

#### **2. Core Functionality**
The calculator app will provide the following functionalities:
1. **Basic Arithmetic Calculations:**  
   - Addition (`+`)  
   - Subtraction (`-`)  
   - Multiplication (`*`)  
   - Division (`/`)
   
2. **Clear Functionality:**  
   - An option to clear the input fields and reset values.

3. **User Interface Design:**  
   - Minimalistic interface with large, visible buttons for operati

Failed to select agent: Agent Failure - Strategy unable to select next agent: User


AgentChatException: Failed to select agent

In [ ]:
import re

async for response in group_chat.invoke():
    if response is None:
        continue

    print(f"{response.role} - {response.name}: {response.content}")
    conversation.append({"role": response.role, "content": response.content})

    # Auto-inject approval if PO is ready
    if response.role == AuthorRole.ASSISTANT and "READY FOR USER APPROVAL" in response.content.upper():
        print("🛑 Waiting for user approval (auto-approving here for demo)...")
        approved_msg = ChatMessageContent(role=AuthorRole.USER, content="APPROVED")
        await group_chat.add_chat_message(approved_msg)
        conversation.append({"role": "user", "content": "APPROVED"})

    if response.role == AuthorRole.USER and "APPROVED" in response.content.upper():
        print("✅ Approved! Saving code and pushing to GitHub (skip in notebook)...")
        break
